### Missing Data

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName('missing_data').getOrCreate()

In [8]:
import os

In [9]:
data_containing_null = os.path.join(os.curdir, 'data', 'ContainsNull.csv')

In [10]:
df = spark.read.csv(data_containing_null, \
                    inferSchema=True, \
                   header=True)

In [12]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [13]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [15]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [17]:
df.na.drop(how='any').show() # other option is all

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [19]:
df.na.drop(subset=['Sales']).show() # column specific null

# df.na.drop(subset=['Sales','Name']).show() # column specific null

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [20]:
df.na.fill('blank value').show()

+----+-----------+-----+
|  Id|       Name|Sales|
+----+-----------+-----+
|emp1|       John| null|
|emp2|blank value| null|
|emp3|blank value|345.0|
|emp4|      Cindy|456.0|
+----+-----------+-----+



In [21]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [22]:
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [23]:
from pyspark.sql.functions import mean

In [25]:
mean_val = df.select(mean(df['Sales'])).collect()

In [26]:
mean_val

[Row(avg(Sales)=400.5)]

In [27]:
mean_sales = mean_val[0][0]

In [28]:
mean_sales

400.5

In [29]:
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Dates and Timestamps

In [30]:
apple_stock_file = os.path.join(os.curdir, 'data', 'appl_stock.csv')

In [48]:
apple_stock = spark.read.csv(apple_stock_file, 
                            inferSchema=True,
                            header=True)

apple_stock.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [34]:
apple_stock.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [36]:
from pyspark.sql.functions import (
    dayofmonth, hour, dayofyear, month, year \
    ,weekofyear, format_number, date_format)

In [39]:
apple_stock.select(dayofmonth(apple_stock['Date']), \
                  weekofyear(apple_stock['Date']) \
                  ,month(apple_stock['Date'])).show()

+----------------+----------------+-----------+
|dayofmonth(Date)|weekofyear(Date)|month(Date)|
+----------------+----------------+-----------+
|               4|               1|          1|
|               5|               1|          1|
|               6|               1|          1|
|               7|               1|          1|
|               8|               1|          1|
|              11|               2|          1|
|              12|               2|          1|
|              13|               2|          1|
|              14|               2|          1|
|              15|               2|          1|
|              19|               3|          1|
|              20|               3|          1|
|              21|               3|          1|
|              22|               3|          1|
|              25|               4|          1|
|              26|               4|          1|
|              27|               4|          1|
|              28|               4|     

In [45]:
# Calculate average closing sales per year

# apple_stock.select(year(apple_stock['Date'])).show()

new_df = apple_stock.withColumn('year', \
                                year(apple_stock['Date']))

evaluated_df = new_df.groupBy('year').mean().select(['year', 'avg(Close)'])

In [47]:
evaluated_df.select(['year', format_number('avg(Close)', 2).alias('average_close')]).show()

+----+-------------+
|year|average_close|
+----+-------------+
|2015|       120.04|
|2013|       472.63|
|2014|       295.40|
|2012|       576.05|
|2016|       104.60|
|2010|       259.84|
|2011|       364.00|
+----+-------------+

